## Please read this if you want the notebook to work


__What this does:__ Get the data from the API and perform basic preprocessing and aggregating

__When to do it:__ Run this module **weekly** to pick up any surveys that were completed. Then run the other notebooks and scripts off the generated data.

#### Tasks

1. Retrieve data from the api and save output as .json\*
2. Adds the 'location-date' column to the dimensional data and survey data
3. Attaches the water_name variable to the survey data
3. Fixes city names that don't match the official name
   1. tells which ones it could not fix
   2. drops records that could not be fixed\*
4. Attaches BFS number to location data
5. Identifies BFS numbers in the beach data that do not match STATPOP or STATENT
   1. drops records where no match is found\*
6. Adds population to the beach data (uses the BFS number)
7. Adds zeros to the results\*
8. Aggregates on the parent_code variable of the code data
9. Saves all that as .csv format and creates a data_directory.json file
   1. the json file is loaded in the template

   

\* notes:

**retrieve data** the endpoints can be changed, the ones given are the root of all the other endpoints

**drop records** there are methods available include new city names and BFS numbers. If you introduce a new location it may 
end up on the list of locations to drop. If that is the case, you can supply the correct information in this workbook and save the data. For STATENT or STATPOP changes see the statent workbook.

**Add zeroes** For this study, if an object was found once, anywhere it is considered as part of the national inventory. Therfore if it is not found/indentified on a survey then this is data we need to track. If an object is part of the national inventory and it was not identified in a survey, the object is added to the survey with a quantity value of zero.

**The survey dimensions valid only for 2020 project** contain the length, area, weights, number of participants and the like. So if you are doing a surface area calculation or wieghts then this is the data you need. The survey dimensions are  keyed by the tuple (location, date), there is one for each survey.



questions or comments: analyst@hammerdirt.ch

In [1]:
import requests
import utilities.utility_functions as ut
import pandas as pd
import os
import csv
import json
import numpy as np

# the local file structure. The resources are located in the corresponding directory.
# the purpose and date should be included in the filename when saving results to output
survey_data, location_data, code_defs, stat_ent, geo_data, output = ut.make_local_paths()

### <span style="color:#008891">Get the current data from the server </span>

In [2]:
def get_the_data(end_points):
    """Takes an array of 2d tuples or arrays ('name', 'url') and
    returns a dictionary of named data objects.

    Used in all notebooks that read or write data.
    """
    data = {}
    for pair in end_points:
        print(pair[1])
        data[pair[0]] = requests.get(pair[1])
    return data


def write_the_data(a_source, a_dir):
    """Writes the response objects (a JSON object) to the provided location.

    Used in notebooks that make an api call
    """
    outPut = []
    with open(a_dir, 'w') as outfile:
        json.dump(a_source.json(), outfile)

def put_the_data_to_local(end_points, here):
    """Gets the data from the provided URL and writes it to the provided location.

    Used in notebooks that make an api call
    """
    the_dict = get_the_data(end_points)
    write_the_data(the_dict, here)
    
def unpack_survey_results(survey_results):
    """Unpacks the surveys-results api-endpoint and adds the location name to each result dict.

    Used in notebooks that make an api call to 'https://mwshovel.pythonanywhere.com/api/surveys/daily-totals/code-totals/swiss/'
    """
    unpacked = []
    for location_data in survey_results:
        location = location_data['location']
        for each_dict in location_data['dailyTotals']:
            each_dict['location']=location
            unpacked.append(each_dict)
    return unpacked

In [3]:
# whatever prefix you are using
prefix = 'https://mwshovel.pythonanywhere.com/api/'

# the resources that you need from the api:
data_i_need = [
    'surveys/daily-totals/code-totals/swiss/',
    'surveys/dim-data/dim-data-list/',    
    'list-of-beaches/swiss/',
    'mlw-codes/list/',    
   
]

# some names for the files
data_names = [
    'daily_code_totals',
    'survey_dims',
    'list_of_beaches',
    'mlw_codes'   
]

# places to store the data
dirs = [
    F"{survey_data}/daily_code_totals.json",
    F"{survey_data}/survey_dims.json",
    F"{location_data}/list_of_beaches.json",
    F"{code_defs}/mlw_codes.json"    
]

data_directory = [*dirs]

# make tuple from the name and the urls
get_this = [(data_names[i],F"{prefix}{x}") for i,x in enumerate(data_i_need)]

# get the data
a_collection_of_objects = get_the_data(get_this)

# write the data to local in .json format
for i,key in enumerate(data_names):
    write_the_data(a_collection_of_objects[key],dirs[i])


https://mwshovel.pythonanywhere.com/api/surveys/daily-totals/code-totals/swiss/
https://mwshovel.pythonanywhere.com/api/surveys/dim-data/dim-data-list/
https://mwshovel.pythonanywhere.com/api/list-of-beaches/swiss/
https://mwshovel.pythonanywhere.com/api/mlw-codes/list/


### <span style="color:#008891"> Unpack the data and save to csv </span>

#### The api is designed to facilitate web output, so there is some prepocessing to do

Before going any further make csv copy of the refreshed data. The survey data needs to be unpacked. The results are grouped by location and then the survey dates and values. To unpack we need to assign the location to each set of values.

*Make loc-date variable*. The loc-date variable does not come off the api. This needs to be done here. There are multiple locations with surveys on the same date. To differentiate between them the location and date need to be combined for each survey record. This is done for the dimensional data as well. 

*Integrate BFS number to beach data*. The BFS number is the key to population and economic data at the municipal level. The BFS number is attached to the beach data by way of the 'city' variable that defines each survey location.

*Attach zeros to each survey*. **In this type of data, how many times something was not seen is just as important as how many times it was seen**.

In [4]:
survey_details = ut.json_file_get(dirs[0])
survey_dims = ut.json_file_get(dirs[1])
beach_data = ut.json_file_get(dirs[2])
code_definitions = ut.json_file_get(dirs[3])

### <span style="color:#008891"> Add loc_date variable to survey and dimensional data </span>

In [5]:
sd_up = ut.unpack_survey_results(survey_details)
sd_df = pd.DataFrame(sd_up)
sd_df['loc_date'] = tuple(zip(sd_df.location, sd_df.date))

In [6]:
sdim_df = pd.DataFrame(survey_dims)
sdim_df['loc_date'] = tuple(zip(sdim_df.location, sdim_df.date))
sdim_df.iloc[1]

survey_key          aare-limmatspitz2020-07-13120
date                                   2020-07-13
length                                        120
area                                       253.00
mac_plast_w                               155.000
mic_plas_w                                  0.033
total_w                                     0.155
est_weight                                    0.0
num_parts_staff                                 1
num_parts_other                                 0
time_minutes                                   90
participants                               ["HD"]
project                                      2020
is_2020                                      True
location                         aare-limmatspitz
loc_date           (aare-limmatspitz, 2020-07-13)
Name: 1, dtype: object

## <span style='color:blue'> Intgerate bfs number </span>

The location data or 'beach data' contains the city of survey location. The city is put in by the surveyor and does not always match what the 'official' city name is.

Here the city names are corrected and then the BFS number is mapped to the beach data with the corrected city name.

There is economic and demographic data availbale for the municipalities in this study. *The statistique structurelle des entreprises* give estimates on employment in different economic categories.

Available at: [STATENT](https://www.bfs.admin.ch/bfs/fr/home/statistiques/industrie-services/entreprises-emplois/structure-economie-entreprises.html)

Population data can be found here: [STATPOP](https://www.bfs.admin.ch/bfs/fr/home/statistiques/population/enquetes/statpop.html)

All of this data is keyed according to the BFS number for each municipality. The BFS number can be found here: [Répertoire officiel des communes de Suisse](https://www.bfs.admin.ch/bfs/fr/home/bases-statistiques/repertoire-officiel-communes-suisse.html)

We are using statent-2017-GMDE and statpop-2018-GMDE in the resources directory.

In [7]:
# set the index of beach data to slug:
beach_data = pd.DataFrame(beach_data).set_index('slug')

In [8]:
# set the index of beach data to slug:
# beach_data = pd.DataFrame(beach_data).set_index('slug')

# get the statent and statpop:
stat_ent_c = pd.read_csv(F"{stat_ent}/stat_ent_corrected.csv").set_index('GDENR')
stat_pop = pd.read_csv(F"{stat_ent}/STATPOP2018_GMDE.csv").set_index('GDENR')

# get the location-bfs keys:
bfs_loc_key = pd.read_csv(F"{stat_ent}/bfs_num.csv").set_index('GDENAME')

# get the dict of beach names to be fixed
# this has the city names we know need to be fixed already
re_names = ut.json_file_get(F"{stat_ent}/not_names.json")

In [9]:
# find city names in beach data that are not in the official names list:
c_names = beach_data.city.unique()
gde_names = bfs_loc_key.index


# these are the city names that need changing for sure:
not_names_keys = list(re_names.keys())

check_me = [x for x in c_names if x not in gde_names]
print(F"These city names should be changed at the server\n{not_names_keys}\n")

# these are the municipalities that we cannot account for:
not_accounted_for = [x for x in check_me if x not in not_names_keys]
print(F"\nThese city names have not been accounted for:\n{not_accounted_for}")

These city names should be changed at the server
['Saint Gingolph', 'St Gallen', 'Jona city', 'Saint Sulpice', 'Illnau Effretikon', 'Zurich', 'biel', 'Cheyres', 'Bremgarten', 'Domat Ems', 'Kradolf Schönenberg', 'Wil', 'Zell', 'Disentis Mustér', 'La Tour de Peilz', 'Küsnacht', 'Hauterive', 'Biel', 'Brienze (BE)', 'Lavey-les-bains']


These city names have not been accounted for:
['Reinach', 'Huningue', 'Büsingen a Hochrhein', 'Selyshche']


### <span style="color:#008891"> If needed update the index of misnamed cities and save to local. </span>

Then use it to update the city names in beach_data, uncomment the code below and insert a key, value pair to the update statement.

In [10]:
# update the renames dictionary
# re_names.update({
#    "Lavey-les-bains":"Lavey-Morcles"
# })
# a_file_name = '{}/{}'.format(my_folders['data'], 'not_names.json')
# ut.push_this_to_json(filename=a_file_name, data=re_names)

# if the above script needed to be run then there are city names to be fixed
# either way the script needs to run to fix the names that we already know about.

def fix_city_names(x, the_names):
    if x in the_names.keys():
        package = the_names[x]
    else:
        package =x
    return package

# replace the old names with the new and delete the old ones
beach_data['city2']= beach_data.city.map(lambda x: fix_city_names(x, re_names))
beach_data.drop('city', axis=1, inplace=True)
beach_data.rename(columns={"city2":"city"}, inplace=True)

# see if their are any names that need to be fixed:
c_names = beach_data.city.unique()

check_me = [x for x in c_names if x not in gde_names]
drop_us = beach_data.loc[beach_data.city.isin(check_me)].index.to_list()

print(F"The following citys could not be reconciled with the register of communes, they are being dropped from the wotking data\n{drop_us}")
print("\nThe original data can always be used by refering to .JSON object in the surveydata folder.\n")
beach_data = beach_data[~beach_data.city.isin(check_me)]

The following citys could not be reconciled with the register of communes, they are being dropped from the wotking data
['birs_reinach_dinuccin', 'rhein_basel_blarerp_1', 'rhein_basel_blarerp_2', 'rhein_laag_jungbluthn', 'untersee_steckborn_siedlerm']

The original data can always be used by refering to .JSON object in the surveydata folder.



### <span style="color:#008891"> At this point add the bfs number to the beach data </span>

In [11]:
beach_data['bfsnum'] = beach_data.city.map(lambda x: bfs_loc_key.loc[x]['GDENR'])
# use that as a map to add the bfs number to the beach data
beach_data.iloc[:5][['location', 'bfsnum']]

,location,bfsnum
slug,,
aabach,Aabach,3338
aare-limmatspitz,Aare Limmatspitz,4029
aare-solothurn-lido-strand,Aare Solothurn Lido Strand,2601
aarezufluss_bern_scheurerk,aarezufluss_bern_scheurerk,351
aare_bern_caveltin,Aare_Bern_CaveltiN,356


#### check for nan values after adding the bfs number

In [12]:
beach_data['bfsnum'].isna().sum()

0

### <span style="color:#008891"> Check the bfs num against statent and stapop bfs numbers </span>

Some time these lists get out of sync, make sure that all the locations have a bfs number that is valid for statent and/or statpop data.

For some locations the city name will not resolve to a BFS number. Those records will be dropped (for now). This is a limited list of locations and none of them are on a lake of interest.

In [13]:
statent_keys = list(stat_ent_c.index)
statpop_keys = list(stat_pop.index)

ent_pop = [x for x in statent_keys if x not in statpop_keys]
pop_ent = [x for x in statpop_keys if x not in statent_keys]
keysweneed = beach_data.bfsnum.unique()

missing_pop_keys = [x for x in keysweneed if x not in statpop_keys]
missing_ent_keys = [x for x in keysweneed if x not in statent_keys]

print(F"These are in statent but not in statpop:\n{ent_pop}\n")
print(F"\nThese are in statpop but not in statent:\n{pop_ent}\n")
print(F"These are the pop keys that do not match beach_data keys:\n{missing_pop_keys}\n")
print(F"These are the stat keys that do not match beach_data keys:\n{missing_ent_keys}\n")

These are in statent but not in statpop:
[132, 133, 217, 222, 624, 865, 875, 878, 3503, 3521, 3522, 3611, 3616, 4113, 6402, 6409, 6410, 6411, 6414, 6415, 6705, 6803]


These are in statpop but not in statent:
[294, 295, 3544]

These are the pop keys that do not match beach_data keys:
[]

These are the stat keys that do not match beach_data keys:
[295]



### <span style="color:#008891"> Chase up any bfs numbers that don't match. </span>

As of january 2021 there is one number: 295 that remains unacounted for.

In [14]:
print(beach_data[beach_data.bfsnum == 295].index.values[0])
drop_us.append(beach_data[beach_data.bfsnum == 295].index.values[0])

sihl_horgen_bucherf


In [15]:
# drop that record from the working data
# add that name to the check me list

beach_data = beach_data[beach_data.bfsnum != 295].copy()
print(F"The BFS num was appended to all records except for the follwoing:\n{drop_us}\nThey have been dropped from the data")

The BFS num was appended to all records except for the follwoing:
['birs_reinach_dinuccin', 'rhein_basel_blarerp_1', 'rhein_basel_blarerp_2', 'rhein_laag_jungbluthn', 'untersee_steckborn_siedlerm', 'sihl_horgen_bucherf']
They have been dropped from the data


### <span style="color:#008891"> Add population to beach data </span>

In [16]:
tot_pop_keyed = stat_pop['B18BTOT'].copy()

beach_data['population'] = beach_data.bfsnum.map(lambda x: tot_pop_keyed[x])
beach_data[['location', 'population']].iloc[:5]

,location,population
slug,,
aabach,Aabach,3710
aare-limmatspitz,Aare Limmatspitz,5414
aare-solothurn-lido-strand,Aare Solothurn Lido Strand,16777
aarezufluss_bern_scheurerk,aarezufluss_bern_scheurerk,133883
aare_bern_caveltin,Aare_Bern_CaveltiN,13054


### <span style="color:#008891"> Add zeroes to results</span>

In [17]:
# go back to the original results and drop the records identified previously
adf = sd_df[~sd_df.location.isin(drop_us)].copy()

# make a list of all the codes that have been identified at least once:
current_codes = adf.code.unique()

# make sure to add the loc_date identifier:
adf['loc_date'] = tuple(zip(adf['location'],adf['date'],))

# get a list of surveys using the loc_date column:
current_surveys = adf.loc_date.unique()

current_surveys[:3]

array([('baby-plage-geneva', '2021-02-10'),
       ('baby-plage-geneva', '2021-01-16'),
       ('baby-plage-geneva', '2020-12-16')], dtype=object)

In [18]:
def insert_zeroes_to_a_group(a_group, current_codes):
    g_codes = a_group.code.unique()
    g_date = a_group.date.unique()
    g_loc = a_group.location.unique()
    missing = [x for x in current_codes if x not in g_codes]
    if len(missing) > 0:
        add_us = []
        for code in missing:            
            add_us.append({'date':g_date[0], 'location':g_loc[0], 'code':code, 'pcs_m':0, 'quantity':0, 'loc_date':(g_loc[0], g_date[0])})
    return a_group.append(add_us)

def insert_zeroes_survey_results(adf, current_surveys, current_codes):
    new_df = pd.DataFrame(columns=adf.columns)
    for the_tup in current_surveys:
        a_group = adf.loc[(adf.loc_date == the_tup)]
        with_zed = insert_zeroes_to_a_group(a_group, current_codes)
        new_df = pd.concat([new_df, with_zed])
    return new_df
        

survey_data_with_zeroes =  insert_zeroes_survey_results(adf, current_surveys, current_codes)

#### <span style="color:#008891">Save the survey data and beach data locally as a csv</span>

Before saving, add the water_body name to the survey results. This saves a step in aggregating regional data later on.

In [19]:
map_water = beach_data.water_name
survey_data_with_zeroes['water_name'] = survey_data_with_zeroes['location'].map(lambda x: map_water[x] )

In [20]:
a_file_name = F"{survey_data}/results_with_zeroes.csv"
data_directory.append(a_file_name)
survey_data_with_zeroes.to_csv(a_file_name, index=False )

In [21]:
another_file_name = F"{survey_data}/dims_data.csv"
data_directory.append(another_file_name)
sdim_df.to_csv(another_file_name, index=False)

In [22]:
location_file = F"{location_data}/beaches_pop_bfs.csv"
data_directory.append(location_file)
beach_data.to_csv(location_file)

In [23]:
code_file = F"{code_defs}/mlw_codes.csv"
data_directory.append(code_file)
code_df = pd.DataFrame(code_definitions)
code_df.to_csv(code_file, index=False)

### <span style="color:#008891"> Aggregate parent codes </span>

All of the codes have a 'parent_code' attribute. The parent_code is the group of several other codes. For example G74 (Extruded polystyrene) is the parent code to: G909 and G910 (extruded polystyrene < or > 25mm).

If you want to aggregate on MLW codes then use the aggregated data. The method to aggregate the results off the API are below

In [24]:
# we can use the data we just finished working with:
aggdf = pd.read_csv(F"{survey_data}/results_with_zeroes.csv")
len(aggdf)

302165

In [25]:
# make a new column called p_code, make that equal to the parent_code column
# this works because the default value of parent_code is "Parent code", so any code
# that does not have a parent code will not aggregate.
code_df['p_code'] = code_df['parent_code']

# set the the value of p_code = to the value we intend on aggregating
# the value of p_code will either be the value of the code itself or the parent_code
# check np.wehre to see how this method works, pandas also has an implementation
code_df['p_code'] = np.where(code_df['p_code'] == 'Parent code', code_df['code'], code_df['p_code'])
print(code_df.columns)
code_df.set_index('code', inplace=True, drop=True)

Index(['code', 'material', 'description', 'source', 'source_two',
       'source_three', 'parent_code', 'direct', 'single_use', 'micro',
       'ospar_code', 'p_code'],
      dtype='object')


In [26]:
print(F"Getting back to code G74, check the parent code of G910\n{code_df.loc['G910']}")

Getting back to code G74, check the parent code of G910
material                                                  Plastic
description     Foamed-plastic for insulation; yellow, pink, b...
source                                               Construction
source_two                                Where does it come from
source_three                                                 none
parent_code                                                   G74
direct                                                      False
single_use                                                  False
micro                                                       False
ospar_code                                                     46
p_code                                                        G74
Name: G910, dtype: object


In [27]:
aggdf['p_code'] = aggdf['code'].map(lambda x: code_df.loc[x].p_code)
aggs = {'date':'first',
        'pcs_m':'sum',
        'quantity':'sum',
        'location':'first',
        'water_name':'first',        
}
aggregated = aggdf.groupby(['loc_date', 'p_code'], as_index=False).aggregate(aggs)
aggregated.rename(columns={'p_code':'code'}, inplace=True)
aggregated.iloc[:3]

,loc_date,code,date,pcs_m,quantity,location,water_name
0,"('aabach', '2020-10-22')",G1,2020-10-22,0.0,0,aabach,Zurichsee
1,"('aabach', '2020-10-22')",G10,2020-10-22,0.0,0,aabach,Zurichsee
2,"('aabach', '2020-10-22')",G100,2020-10-22,0.0,0,aabach,Zurichsee


### <span style="color:#008891"> Save the aggregated values and post data directory to resources </span>

In [28]:
second_to_last_file_name = F"{survey_data}/results_with_zeroes_aggregated_parent.csv"
print(second_to_last_file_name)
data_directory.append(second_to_last_file_name )
aggregated.to_csv(second_to_last_file_name, index=False)

resources/surveydata/results_with_zeroes_aggregated_parent.csv


In [29]:
the_last_file_name = "resources/data_directory.json"
ut.push_this_to_json(filename=the_last_file_name, data=data_directory)

putting: resources/data_directory.json


hopefully this just worked for you.

If not contact analyst@hammerdirt.ch

